# There is a lack of data with label of "0" left_side > center > right_side
# The approach to create new data with label of "0" is proposed below

In [55]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join('..'))) # imports top level

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import display
from main.permutations import Permutations
from collections import defaultdict

In [56]:
def validation_stacking_bagging(
    models:dict,
    X_val_raw,
    y_val = None,
    averaging:bool = False
):
    res_dict = defaultdict(list)

    for key, model in models.items():
        # res_dict[key] = defaultdict()
        for k, v in model.items():
    
            base = v["base"]
            X_val = base["ct_base"].transform(X_val_raw)
            X_val_base = X_val.copy()
            
        
            # print(X_val_raw)
            for kb, bvals in base["models"].items():
                if kb == "bagging" and len(bvals) > 0:
                    for bag_names, bag_mods in base["models"].get("bagging").items():
                        for bnum, bag_mod in enumerate(bag_mods):
                            X_val_base.loc[:, [f"{bag_names}_{bnum}"]] = bag_mod.predict(X_val)
                
                elif kb != "bagging":
                    X_val_base.loc[:, kb] = bvals.predict(X_val)
                
            X_meta = X_val_base

            # X_meta = X_meta.loc[
            #     :,
            #     list(set(X_meta).difference(set(X_val)))
            # ]
            
            X_meta = X_meta.reindex(sorted(X_meta), axis=1)

            res_dict[k].append( v["meta"].predict(X_meta) )
    
    res_arr = np.array(list(res_dict.values())).mean(axis=1)
    
    res = pd.DataFrame(data=res_arr).T
    res.columns = res_dict.keys()
    # print(res_dict)
    # print(res)
    if y_val is not None:
        print(mean_absolute_error(
            res,
            y_val
        ))
    return res


In [57]:
from joblib import load
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

In [58]:
reg_model = load("stck_bag_non_unifor_v1.1.joblib")

In [59]:
df_to_aug = pd.read_excel("input.xlsx", sheet_name="no_burnup")

In [60]:
d = display.Display(df=df_to_aug)
d.quick_processing()

The columns with prefix  are not in DataFrame


In [61]:
d.df

,Name,pmar,"p, %","p, beff",7-6_s,7-5_s,7-4_s,7-3_s,6-6_s,6-5_s,...,average_t,right_center_side_t,left_center_side_t,right_side_t,left_side_t,center_t,qul_t,qur_t,qll_t,qlr_t
0,112,1.07861,7.297184,9.295775,1.051558,0.790403,1.013165,1.037370,0.912569,1.189814,...,282.50,282.5,282.5,286.0,279.0,282.5,279.0,279.0,286.0,286.0
1,114,1.08357,7.774369,9.903655,0.648665,0.664725,1.083983,0.954085,0.795975,0.990852,...,282.50,286.0,279.0,286.0,272.0,286.0,279.0,286.0,279.0,286.0
2,114_7-6,1.07174,6.755688,8.605972,1.114553,0.746294,1.187454,1.001877,0.900434,1.090843,...,282.50,286.0,279.0,286.0,272.0,286.0,279.0,286.0,279.0,286.0
3,114_fake11,1.08923,8.368326,10.660287,0.641218,0.642833,1.041846,0.908672,0.806680,0.976815,...,282.50,286.0,279.0,286.0,272.0,286.0,279.0,286.0,279.0,286.0
4,114_fake12,1.10397,9.594128,12.221820,0.676469,1.019903,1.181528,0.942797,0.843354,0.994229,...,282.50,286.0,279.0,286.0,272.0,286.0,279.0,286.0,279.0,286.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,127_fake1,1.08339,7.803036,9.940173,0.971585,1.088872,0.882367,0.840697,0.930700,0.783125,...,284.25,286.0,282.5,286.0,279.0,286.0,279.0,286.0,286.0,286.0
170,127_fake3,1.08216,7.698123,9.806526,0.973637,1.127915,0.919568,1.304821,0.910114,0.785559,...,284.25,286.0,282.5,286.0,279.0,286.0,279.0,286.0,286.0,286.0
171,127_fake4,1.07498,7.080913,9.020272,0.964947,1.111812,0.898927,1.274349,0.905553,0.774215,...,284.25,286.0,282.5,286.0,279.0,286.0,279.0,286.0,286.0,286.0
172,127_fake5,1.07546,7.122432,9.073162,1.008633,1.128648,1.221129,0.924833,0.934781,0.793533,...,284.25,286.0,282.5,286.0,279.0,286.0,279.0,286.0,286.0,286.0


In [62]:
new_dfs = []

aug_indexes = [np.random.randint(0, len(df_to_aug)) for i in range(5)]
for i in aug_indexes:
    p = Permutations(df = df_to_aug.loc[:,
        [
            "Name",
            *d.CORE_PARTS["burnup"]["ALL_CELLS"],
            *d.CORE_PARTS["fuel_type"]["ALL_CELLS"],
            *d.CORE_PARTS["left"]["ALL_CELLS"]
        ]
    ], ind=i)

    fr, to = p._random_permutations(3)
    print(fr, to)
    dfs = p.make_permutations(fr, to, chain_like=True)
    df = pd.concat(dfs, axis=0).reset_index().drop("index", axis=1)
    # print(df)
    # print(validation_stacking_bagging(reg_model, df))
    df = pd.concat([df, validation_stacking_bagging(reg_model, df)], axis=1)
    new_dfs.append(df)
df_aug = pd.concat(new_dfs, axis=0).reset_index().drop("index", axis=1)

['3-3', '2-4', '6-5'] ['6-3', '2-3', '3-3']
Detected among six tubes ['3-3', '6-3', '6-5'] Detected among eight tubes ['2-3', '2-4']
['4-6', '7-6', '2-4'] ['7-4', '7-4', '7-5']
Detected among six tubes [] Detected among eight tubes ['2-4', '7-5', '4-6', '7-6', '7-4']
['2-5', '6-4', '3-5'] ['7-5', '6-6', '6-3']
Detected among six tubes ['6-4', '6-3', '6-6', '3-5'] Detected among eight tubes ['7-5', '2-5']
['6-4', '4-3', '7-4'] ['3-5', '7-5', '5-3']
Detected among six tubes ['6-4', '3-5'] Detected among eight tubes ['4-3', '7-5', '5-3', '7-4']
['2-6', '7-4', '6-4'] ['2-5', '7-3', '3-5']
Detected among six tubes ['6-4', '3-5'] Detected among eight tubes ['7-3', '2-5', '2-6', '7-4']


In [63]:
df_aug

,Name,7-6_b,7-5_b,7-4_b,7-3_b,6-6_b,6-5_b,6-4_b,6-3_b,5-6_b,...,4-6_s,4-3_s,3-6_s,3-5_s,3-4_s,3-3_s,2-6_s,2-5_s,2-4_s,2-3_s
0,121_e1,57.12,16.14,25.92,150.18,166.0225,76.0020,141.0330,149.9635,83.1570,...,1.162786,1.170966,0.661050,0.924500,1.130754,0.782096,1.031676,0.827086,1.012486,0.777089
1,121_e1,57.12,16.14,25.92,150.18,166.0225,76.0020,141.0330,149.9635,83.1570,...,1.167418,1.185022,0.657354,0.911917,1.147734,0.783957,1.016998,0.826573,0.777383,1.143394
2,121_e1,57.12,16.14,25.92,150.18,166.0225,157.3570,141.0330,149.9635,83.1570,...,1.162553,1.206945,0.687824,0.940553,1.182642,1.148200,1.031743,0.869825,0.830766,1.193333
3,117_fake_e23,184.38,168.66,171.99,174.60,140.7945,19.9280,95.4265,116.9445,32.0650,...,0.989463,1.422453,0.771757,1.089844,1.429861,1.077583,0.690699,0.921057,0.774942,1.202935
4,117_fake_e23,171.99,168.66,184.38,174.60,140.7945,19.9280,95.4265,116.9445,32.0650,...,0.988583,1.421581,0.773089,1.088157,1.423505,1.076182,0.689540,0.920257,0.775108,1.167842
5,117_fake_e23,171.99,166.92,184.38,174.60,140.7945,19.9280,95.4265,116.9445,32.0650,...,0.988349,1.419660,0.772687,1.087134,1.417918,1.072199,0.688794,0.919369,0.765314,1.166871
6,127_5-6,39.69,123.03,124.23,149.04,87.9800,154.8130,69.8805,86.3900,0.0000,...,0.675849,1.150847,0.944333,1.182894,0.814618,1.092231,1.025081,1.116236,1.143876,0.819111
7,127_5-6,39.69,123.03,124.23,149.04,69.8805,154.8130,87.9800,86.3900,0.0000,...,0.677315,1.148966,0.947819,1.183054,0.816170,1.096825,1.023253,1.119457,1.145961,0.819092
8,127_5-6,39.69,123.03,124.23,149.04,69.8805,154.8130,87.9800,63.4410,0.0000,...,0.671327,1.153192,0.936475,1.090482,0.806401,1.093610,1.011886,1.107362,1.134708,0.812640
9,123_2-3,162.51,75.33,45.48,36.06,18.5235,93.0945,141.9870,168.4340,99.2955,...,0.909378,1.125272,1.169446,0.833080,1.115800,0.768784,0.728091,1.081956,1.104322,1.053096


In [64]:
d2 = display.Display(df=df_aug)
d2.quick_processing()

The columns with prefix  are not in DataFrame


In [65]:
d2.df

,Name,7-6_b,7-5_b,7-4_b,7-3_b,6-6_b,6-5_b,6-4_b,6-3_b,5-6_b,...,average_t,right_center_side_t,left_center_side_t,right_side_t,left_side_t,center_t,qul_t,qur_t,qll_t,qlr_t
0,121_e1,57.12,16.14,25.92,150.18,166.0225,76.0020,141.0330,149.9635,83.1570,...,284.25,286.0,282.5,286.0,279.0,286.0,279.0,286.0,286.0,286.0
1,121_e1,57.12,16.14,25.92,150.18,166.0225,76.0020,141.0330,149.9635,83.1570,...,284.25,286.0,282.5,286.0,279.0,286.0,279.0,286.0,286.0,286.0
2,121_e1,57.12,16.14,25.92,150.18,166.0225,157.3570,141.0330,149.9635,83.1570,...,284.25,286.0,282.5,286.0,279.0,286.0,279.0,286.0,286.0,286.0
3,117_fake_e23,184.38,168.66,171.99,174.60,140.7945,19.9280,95.4265,116.9445,32.0650,...,282.50,286.0,279.0,286.0,272.0,286.0,279.0,286.0,279.0,286.0
4,117_fake_e23,171.99,168.66,184.38,174.60,140.7945,19.9280,95.4265,116.9445,32.0650,...,282.50,286.0,279.0,286.0,272.0,286.0,279.0,286.0,279.0,286.0
5,117_fake_e23,171.99,166.92,184.38,174.60,140.7945,19.9280,95.4265,116.9445,32.0650,...,282.50,286.0,279.0,286.0,272.0,286.0,279.0,286.0,279.0,286.0
6,127_5-6,39.69,123.03,124.23,149.04,87.9800,154.8130,69.8805,86.3900,0.0000,...,284.25,286.0,282.5,286.0,279.0,286.0,279.0,286.0,286.0,286.0
7,127_5-6,39.69,123.03,124.23,149.04,69.8805,154.8130,87.9800,86.3900,0.0000,...,284.25,286.0,282.5,286.0,279.0,286.0,279.0,286.0,286.0,286.0
8,127_5-6,39.69,123.03,124.23,149.04,69.8805,154.8130,87.9800,63.4410,0.0000,...,284.25,286.0,282.5,286.0,279.0,286.0,279.0,286.0,286.0,286.0
9,123_2-3,162.51,75.33,45.48,36.06,18.5235,93.0945,141.9870,168.4340,99.2955,...,284.25,286.0,282.5,286.0,279.0,286.0,279.0,286.0,286.0,286.0


In [56]:
ct_c = ColumnTransformer(
    [
        ("", StandardScaler(), [*d.CORE_PARTS["burnup"]["ALL_CELLS"], "average"])
    ]
).set_output(transform="pandas")

In [57]:
to_aug = ct_c.fit_transform(df_to_aug)

In [58]:
aug_pred = reg_model.predict(to_aug)

In [59]:
df_export = pd.DataFrame(data=aug_pred, columns=[*d.CORE_PARTS["coef"]["ALL_CELLS"], "p, %"])
df_export

,7-6_s,7-5_s,7-4_s,7-3_s,6-6_s,6-5_s,6-4_s,6-3_s,5-6_s,5-3_s,...,4-3_s,3-6_s,3-5_s,3-4_s,3-3_s,2-6_s,2-5_s,2-4_s,2-3_s,"p, %"
0,0.565876,0.820930,1.087865,1.054655,1.123758,1.156413,1.253120,0.793886,0.863450,1.446379,...,1.162001,1.189835,1.139019,1.158190,0.802930,1.062857,0.884167,0.780588,0.953298,8.244485
1,0.531745,0.812096,1.093366,1.027055,1.026678,1.180207,0.844621,1.361870,0.849430,1.472135,...,1.187574,1.073690,0.751294,1.176024,1.327681,0.737088,1.069922,0.804204,0.980798,8.220525
2,1.056925,0.836170,1.034927,1.038035,1.089801,1.141639,0.829446,1.335565,0.862660,1.451969,...,1.129836,1.059644,0.744101,1.059563,1.256371,0.733623,1.010967,1.131630,0.853027,8.710929
3,1.083016,0.830244,0.890811,0.864130,0.900703,1.202408,1.262872,0.991982,0.982184,0.920859,...,0.959775,0.898984,1.200042,1.231426,0.983869,1.067849,0.935151,0.808938,0.803314,5.677853
4,0.651002,0.782337,0.953739,1.043829,0.870733,0.927598,1.350601,1.224838,0.804117,1.190859,...,1.546579,0.919336,1.191501,0.946079,1.191680,0.709231,0.875849,0.810694,1.006912,5.790400
5,0.683654,0.838119,1.111719,1.262697,0.869851,0.821684,1.323747,1.195434,0.808757,1.136669,...,1.443866,0.864250,1.181615,0.894945,1.123925,0.710281,0.978176,0.815694,1.114415,6.299949
6,0.678828,1.180481,1.123468,0.819695,0.887237,0.832009,1.220415,1.159180,1.286271,1.110942,...,1.400373,0.842610,1.124445,0.860576,1.084166,0.704057,0.950023,1.111490,0.767078,7.764674
7,0.662899,1.184929,1.138620,0.777525,0.887528,0.820999,1.205607,1.122339,1.277610,1.113825,...,1.350918,0.848283,1.121890,0.804294,1.064953,0.687950,0.997723,1.088377,0.778662,6.469919
8,1.035205,0.844688,1.099567,0.806350,0.901621,0.816099,1.239331,1.136272,1.292621,1.144448,...,1.384426,0.857764,1.166825,0.864354,1.068832,0.973867,0.954048,0.798093,0.754627,8.004233
9,1.063775,0.842056,1.101465,0.809714,0.930180,0.806099,1.235661,1.124962,1.318487,1.124486,...,1.008347,0.907672,1.176843,0.832609,1.057373,1.008272,0.950713,0.801656,0.753194,7.633403


In [60]:
df_export.to_excel("input_augmented.xlsx")

In [61]:
d.display(df_export.iloc[-2][:-1], dt="coef")

,6,5,4,3
7,1.046475,1.116171,0.768605,0.835109
6,0.700906,1.210597,0.975799,0.851364
5,1.123550,nan,nan,1.300808
4,1.164344,nan,nan,1.170389
3,0.717817,0.929155,1.253491,0.826129
2,0.672561,1.068861,1.080880,0.887624
